In [193]:
input = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k"""

In [208]:
with open('day7.txt', 'r') as f:
    input = f.read()

In [209]:
class Path:
    def __init__(self, typ, name, parent):
        self.typ = typ
        self.name = name
        self.parent = parent
        self.children = []
    
    def add(self, p: Path):
        self.children.append(p)
        
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return f"{self.name} ({self.typ})"
    
    def __eq__(self, other): 
        return self.typ == other.typ and self.name == other.name and self.parent == other.parent
    
    def render(self, depth=0):
        print(f"{' '*depth}- {self}")
        for c in self.children:
            c.render(depth + 1)
            
    @staticmethod
    def walk(node):
        for n in node.children:
            yield n
            for c in Path.walk(n):
                yield c

class Directory(Path):
    def __init__(self, name, parent):
        super().__init__('dir', name, parent)
    
    @property
    def size(self):
        return sum([ x.size for x in self.children ])
        
class File(Path):
    def __init__(self, size, name, parent):
        super().__init__('file', name, parent)
        self._size = int(size)
        
    def __str__(self):
        return f"{self.name} ({self.typ}, size={self.size})"
    
    @property
    def size(self):
        return self._size

In [210]:
def parse_input(lines, node):
    #print(f"at {node}")
    for i, line in enumerate(lines):
        #print(f"\t{line}")
        if line[0] == "$":
            cmd = line.split()[1:]
            if cmd[0] == "ls":
                continue
            elif cmd[0] == "cd":
                if cmd[1] == '..':
                    return parse_input(lines[i+1:], node.parent)
                d = Directory(cmd[1], node)
                for f in node.children:
                    if f == d:
                        return parse_input(lines[i+1:], f)
        elif line.startswith("dir"):
            node.add(Directory(line.split()[-1], node))
        else:
            node.add(File(*line.split(), parent=node))
root = Directory('/', None)
parse_input(input.split('\n')[1:], root)

In [211]:
#root.render()

In [220]:
root.size

46975962

In [213]:
total = 0
for n in Path.walk(root):
    if n.typ == "dir" and n.size < 100000:
        total += n.size
total

1334506

In [221]:
needed = 30000000
unused = 70000000 - root.size
could_delete = []
for n in Path.walk(root):
    if n.typ == "dir":
        #print(f"{(root.size - n.size)}")
        if (root.size - n.size) < needed:
            could_delete.append(n)
could_delete

[dzbfsf (dir), tllwm (dir), qjnnw (dir)]

[dzbfsf (dir), tllwm (dir), qjnnw (dir)]

In [219]:
[ x.size for x in could_delete ]

[19354558, 18659329, 17459970]

In [218]:
smallest = could_delete[0]
for r in could_delete[1:]:
    if r.size < smallest.size:
        smallest = r
smallest.size

17459970